In [1]:
import pgbquery as bq
import bcolz as bz
import numpy as np
import pandas as pd
import os
import datetime
import operator
from multicorn import Qual
from sklearn.feature_extraction import DictVectorizer as DV

initiate the bcolz table

In [14]:
test = bq.ctable.create_ctable(test, rootdir='data/test')

TypeError: unbound method create_ctable() must be called with ctable instance as first argument (got ctable instance instead)

append some data

In [3]:
test = bz.ctable([bz.carray(['a', 'b', 'c']), 
                  bz.carray([1,2,3]), 
                  bz.carray([datetime.date(2016,1,2), datetime.date(2016,1,3), datetime.date(2016,1,4)])
                 ], ['letter', 'number', 'date'])

write data from memory to disk

In [ ]:
# test.addcol(['letter a', 'letter b', 'letter c'], 'desc')
#test.delcol('nulls')
#test.addcol([True, True, False], name='booleans')
c = bz.carray([1,np.nan, None], dtype='float64')
test.addcol(c, name='nulls')

load table to memory from disk

In [2]:
test = bq.open('/home/ralbright/data/test', mode='a')
test.dtype

dtype([('letter', 'S1'), ('number', '<i8'), ('date', 'O'), ('desc', 'S8'), ('floats', '<f8'), ('booleans', '?'), ('nulls', '<f8')])

In [3]:
test

ctable((3,), [('letter', 'S1'), ('number', '<i8'), ('date', 'O'), ('desc', 'S8'), ('floats', '<f8'), ('booleans', '?'), ('nulls', '<f8')])
  nbytes: 126; cbytes: 96.52 KB; ratio: 0.00
  cparams := cparams(clevel=5, shuffle=True, cname='blosclz')
  rootdir := '/home/ralbright/data/test'
[('a', 1, datetime.date(2016, 1, 2), 'letter a', 56.43, True, 1.0)
 ('b', 2, datetime.date(2016, 1, 3), 'letter b', 354767327.34, True, nan)
 ('c', 3, datetime.date(2016, 1, 4), 'letter c', -589574.474567347, False, nan)]

In [18]:
c

carray((6,), float64)
  nbytes: 48; cbytes: 16.00 KB; ratio: 0.00
  cparams := cparams(clevel=5, shuffle=True, cname='blosclz')
[  1.   2.   3.  nan  nan  nan]

In [4]:
def execute(quals, columns):
    filters = []
    for qual in quals:
        
       if not qual.is_list_operator:
            if type(qual.value) == str:
                value = "'%s'" % qual.value
            else:
                value = qual.value
            filters.append((str(qual.field_name), str(qual.operator), str(value)))
       else:
            op = qual.operator[0]
            optype = qual.operator[1]
            if op in ('=', '==') and optype == ANY:
                operator = 'IN'
            if op in ('!=', '<>') and optype == ANY:
                operator = 'NOT IN'
                filters.append((qual.fieldname, operator, qual.value))
    if len(quals) > 0:            
        boolarr = test.where_terms(filters)
        for line in test.where(boolarr, outcols=test.names):
            yield line
    else:
        for line in test.__iter__():
            yield line

In [9]:
line = execute([Qual('nulls', 'is not', 'null')], test.names)

In [12]:
line.next()

row(letter='c', number=3, date=datetime.date(2016, 1, 4), desc='letter c', floats=-589574.474567347, booleans=False, nulls=nan)

In [7]:
cols=
cols[0]= bz.zeros(0, 'S25')
cols[1] = bz.zeros(0, 'f8')


SyntaxError: invalid syntax (<ipython-input-7-1d26738a792b>, line 1)

In [3]:
mask = test.where_terms([('nulls', 'is not', "None")])

In [4]:
w = test.where(mask)

In [7]:
w.next()

row(letter='c', number=3, date=datetime.date(2016, 1, 4), desc='letter c', floats=-589574.474567347, booleans=False, nulls=nan)

In [8]:
from pgbquery.operatorFunctions import opMap
print opMap


{'gt': <built-in function gt>, 'is not': <function notnull at 0x7f786be25488>, '~~': <function likeSearch at 0x7f786be25230>, 'is': <function isnull at 0x7f786be25410>, 'ne': <built-in function ne>, 'ge': <built-in function ge>, '!~~*': <function notLikeSearch_i at 0x7f786be25398>, 'eq': <built-in function eq>, '>&': <function leftBounded at 0x7f786be17ed8>, 'le': <built-in function le>, '~~*': <function likeSearch_i at 0x7f786be252a8>, '!~~': <function notLikeSearch at 0x7f786be25320>, 'ilike': <function likeSearch_i at 0x7f786be252a8>, 'similar to': <function regexSearch at 0x7f786be25050>, 'lt': <built-in function lt>, 'not similar to': <function notRegexSearch at 0x7f786be25140>, '&<': <function rightBounded at 0x7f786be17e60>, '~*': <function regexSearch_i at 0x7f786be250c8>, '&&': <function overlap at 0x7f786be17f50>, 'not ilike': <function likeSearch_i at 0x7f786be252a8>, '!~': <function notRegexSearch at 0x7f786be25140>, 'like': <function likeSearch at 0x7f786be25230>, 'not lik

In [8]:
from pgbquery.operatorFunctions

['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__enter__',
 '__exit__',
 '__format__',
 '__getattribute__',
 '__getitem__',
 '__hash__',
 '__init__',
 '__iter__',
 '__len__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_stats',
 '_iter',
 '_where',
 'addcol',
 'aggregate_groups',
 'append',
 'cache_factor',
 'cache_valid',
 'cbytes',
 'copy',
 'cparams',
 'create_agg_ctable',
 'create_ctable',
 'delcol',
 'dtype',
 'eval',
 'factorize_groupby_cols',
 'flush',
 'free_cachemem',
 'fromdataframe',
 'fromhdf5',
 'groupby',
 'is_in_ordered_subgroups',
 'iter',
 'make_group_index',
 'mkdir_rootdir',
 'names',
 'nbytes',
 'ndim',
 'open_ctable',
 'resize',
 'shape',
 'size',
 'todataframe',
 'tohdf5',
 'trim',
 'unique',
 'where',
 'where_pgterms',
 'where_terms',
 'whereblocks']

In [1]:
load_ext Cython

In [2]:
%%cython -a
import re2
import operator
dir(operator)
print operator.eq(1,2)

ImportError: libre2.so.0: cannot open shared object file: No such file or directory